# Install libraries

In [1]:
!pip install Whoosh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 25.9 MB/s eta 0:00:00


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


from whoosh.index import create_in
from whoosh.fields import *
from whoosh.analysis import StandardAnalyzer

from whoosh import qparser
from whoosh import scoring
import whoosh.index as index

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
import os

def read_text_files(link):

  directory = link

  text_files = []

  for filename in os.listdir(directory):
      if filename.endswith(".txt"):
          file_path = os.path.join(directory, filename)
          with open(file_path, 'r') as file:
              content = file.read()
              text_files.append(content)
  return text_files

# Read Cranfield

In [4]:
# Cranfiel
import gdown
!gdown 1D48c_crSg6ia_OqZ5HoJmVmDLgn-iqq5
!gdown 1g0JYIiu-l6dSo5BpOuAeem2rVCkxNHjE

Downloading...
From: https://drive.google.com/uc?id=1D48c_crSg6ia_OqZ5HoJmVmDLgn-iqq5
To: /content/Cranfield.zip
100% 851k/851k [00:00<00:00, 36.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1g0JYIiu-l6dSo5BpOuAeem2rVCkxNHjE
To: /content/TEST.zip
100% 49.9k/49.9k [00:00<00:00, 61.4MB/s]


In [5]:
!unzip 'Cranfield.zip'

Archive:  Cranfield.zip
   creating: Cranfield/
  inflating: Cranfield/1.txt         
  inflating: Cranfield/10.txt        
  inflating: Cranfield/100.txt       
  inflating: Cranfield/1000.txt      
  inflating: Cranfield/1001.txt      
  inflating: Cranfield/1002.txt      
  inflating: Cranfield/1003.txt      
  inflating: Cranfield/1004.txt      
  inflating: Cranfield/1005.txt      
  inflating: Cranfield/1006.txt      
  inflating: Cranfield/1007.txt      
  inflating: Cranfield/1008.txt      
  inflating: Cranfield/1009.txt      
  inflating: Cranfield/101.txt       
  inflating: Cranfield/1010.txt      
  inflating: Cranfield/1011.txt      
  inflating: Cranfield/1012.txt      
  inflating: Cranfield/1013.txt      
  inflating: Cranfield/1014.txt      
  inflating: Cranfield/1015.txt      
  inflating: Cranfield/1016.txt      
  inflating: Cranfield/1017.txt      
  inflating: Cranfield/1018.txt      
  inflating: Cranfield/1019.txt      
  inflating: Cranfield/102.txt       
  

In [6]:
!unzip 'TEST.zip'

Archive:  TEST.zip
  inflating: query.txt               
   creating: RES/
  inflating: RES/1.txt               
  inflating: RES/10.txt              
  inflating: RES/100.txt             
  inflating: RES/101.txt             
  inflating: RES/102.txt             
 extracting: RES/103.txt             
  inflating: RES/104.txt             
  inflating: RES/105.txt             
  inflating: RES/106.txt             
  inflating: RES/107.txt             
  inflating: RES/108.txt             
  inflating: RES/109.txt             
  inflating: RES/11.txt              
  inflating: RES/110.txt             
  inflating: RES/111.txt             
 extracting: RES/112.txt             
  inflating: RES/113.txt             
  inflating: RES/114.txt             
  inflating: RES/115.txt             
  inflating: RES/116.txt             
 extracting: RES/117.txt             
  inflating: RES/118.txt             
 extracting: RES/119.txt             
  inflating: RES/12.txt              
  inflating: 

In [7]:
# Loading the queries of Cranfield
def load_queries(file_path):
    queries = {}
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            key, value = line.strip().split('\t')
            queries[int(key)] = value
    return queries

queries = load_queries('/content/query.txt')
queries

{1: 'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .',
 2: 'what are the structural and aeroelastic problems associated with flight of high speed aircraft .',
 3: 'what problems of heat conduction in composite slabs have been solved so far .',
 4: 'can a criterion be developed to show empirically the validity of flow solutions for chemically reacting gas mixtures based on the simplifying assumption of instantaneous local chemical equilibrium .',
 5: 'what chemical kinetic system is applicable to hypersonic aerodynamic problems .',
 6: 'what theoretical and experimental guides do we have as to turbulent couette flow behaviour .',
 7: 'is it possible to relate the available pressure distributions for an ogive forebody at zero angle of attack to the lower surface pressures of an equivalent ogive forebody at angle of attack .',
 8: 'what methods -dash exact or approximate -dash are presently available for predicting body pressures at

In [8]:
# Reading the Cranfield's documents
import os

directory = '/content/Cranfield'  # Update with the path to your text files
file_extension = '.txt'  # Update with the file extension of your text files

text_files = [file for file in os.listdir(directory) if file.endswith(file_extension)]

data_dict = {}

for file_name in text_files:
    file_path = os.path.join(directory, file_name)
    with open(file_path, 'r') as file:
        contents = file.read()
        key = int(os.path.splitext(file_name)[0])  # Remove the file extension from the key
        data_dict[key] = contents

# Now you have the contents of each text file stored in the 'data_dict' dictionary

sorted_data_dict = dict(sorted(data_dict.items()))

In [9]:
## Loading the 'Gold' result
import os

folder_path = '/content/RES'
trueResult = {}

for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as file:
            lines = file.readlines()
            second_column = [int(line.split()[1]) for line in lines]
            key = int(file_name[:-4])  # Remove the '.txt' extension and convert to int
            trueResult[key] = second_column

trueResult = dict(sorted(trueResult.items()))

# Functions

In [10]:
# Trang ver of AP/mAP by returning just the main mAP value of all 225 queries

def precision(y_true, y_pred):
  prec_list = []
  rec_list = []
  count = 0
  for i in range(len(y_pred)):
    if y_pred[i] in y_true:
      count += 1
      prec_list.append(count/(i+1))
      rec_list.append(count/len(y_true))
  return prec_list, rec_list
# list prec nội suy nè
def prec_in(prec_list, rec_list):
  prec_list_in = []
  for i in range(11):
    rec = i/10
    # prec_rec = max(rec->all)
    # lấy max từ prec_rec_in
    prec_rec_in = next((x[0] for x in enumerate(rec_list) if x[1] >= rec),-1)
    # print('rec_in:', prec_rec_in)
    prec_rec = 0
    if prec_rec_in != -1:
      prec_rec = max(prec_list[prec_rec_in:])
    prec_list_in.append(prec_rec)
  return prec_list_in

In [11]:
def MAP(list_y_true, list_y_pred, noisuy=True):
    common_keys = set(list_y_true.keys()).intersection(set(list_y_pred.keys()))
    result_dict = {}

    if noisuy:
        prec_in_data = {}
        for key in common_keys:
            prec_list, rec_list = precision(list_y_true[key], list_y_pred[key])
            prec_in_list = prec_in(prec_list, rec_list)
            # print('prec_list', prec_list)
            # print('rec_list', rec_list)
            # print('prec_list_in', prec_in_list)
            result_dict[key] = sum(prec_in_list) / len(prec_in_list)
            prec_in_data[key] = prec_in_list
        return prec_in_data, result_dict, sum(result_dict.values()) / len(result_dict.values())
    else:
        for key in common_keys:
            prec_list, rec_list = precision(list_y_true[key], list_y_pred[key])
            if len(prec_list) != 0:
                result_dict[key] = sum(prec_list) / len(prec_list)
            else:
                result_dict[key] = 0

    return result_dict, sum(result_dict.values()) / len(result_dict.values())

In [12]:
def count_P_R(list_y_true, list_y_pred):
  common_keys = set(list_y_true.keys()).intersection(set(list_y_pred.keys()))
  result_P_dict = {}
  result_R_dict = {}
  for key in common_keys:
      count = 0
      for i in range(len(list_y_pred[key])):
        if list_y_pred[key][i] in list_y_true[key]:
          count += 1
      if len(list_y_pred[key])!=0:
        result_P_dict[key] = count / len(list_y_pred[key])
      else:
        result_P_dict[key] = 0
      result_R_dict[key] = count / len(list_y_true[key])
  return result_P_dict, result_R_dict, sum(result_P_dict.values()) / len(result_P_dict.values()), sum(result_R_dict.values()) / len(result_R_dict.values())

In [13]:
def trec_11(prec_in_data):
  trec_11 = {}
  for i in range(11):
    trec_11[i/10] = 0
    for j in prec_in_data:
      trec_11[i/10]+=prec_in_data[j][i]/len(prec_in_data)
  return trec_11

In [14]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
import string

def preprocessQueryStemmingAnalyzer(query_text):
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))

    # Lowercase the query text
    query_text = query_text.lower()

    # Remove non-alphanumeric characters
    query_text = re.sub(r'[^a-z0-9\s]', '', query_text)

    # Tokenize the query text
    words = word_tokenize(query_text)

    # Process each word in the query
    processed_words = []
    for word in words:
        if word not in stop_words and word.isdigit() == False and word.isnumeric() == False and word.isalnum():
            stemmed_word = stemmer.stem(word)
            processed_words.append(stemmed_word)

    # Join the processed words into a single string
    processed_query = ' '.join(processed_words)

    return processed_query


In [15]:
def preprocessQueryWithStopwords(query_text):
    stemmer = PorterStemmer()

    # Lowercase the query text
    query_text = query_text.lower()

    # Remove non-alphanumeric characters
    query_text = re.sub(r'[^a-z0-9\s]', '', query_text)

    # Tokenize the query text
    words = word_tokenize(query_text)

    # Process each word in the query
    processed_words = []
    for word in words:
        stemmed_word = stemmer.stem(word)
        processed_words.append(stemmed_word)

    # Join the processed words into a single string
    processed_query = ' '.join(processed_words)

    return processed_query

# Stem, Remove Stop

In [16]:
# import shutil

# shutil.rmtree('/content/indCranfield')

In [17]:
!mkdir /content/indCranfieldPSRemoveStop

In [18]:
from whoosh.analysis import StandardAnalyzer, StopFilter
from whoosh.analysis import StemmingAnalyzer

import time

start_time = time.time()

schema = Schema(docid=STORED(), content=TEXT(stored=True, analyzer=StandardAnalyzer()))
ix = create_in("indCranfieldPSRemoveStop", schema)
writer = ix.writer()

for i in range(1, len(sorted_data_dict)):
    writer.add_document(docid="{}".format(i + 1), content=sorted_data_dict[i])

writer.commit()

total_timeStemRemoveStop = time.time() - start_time
print("Total execution time:", total_timeStemRemoveStop)

Total execution time: 3.7629919052124023


In [19]:
!ls /content/indCranfieldPSRemoveStop

_MAIN_1.toc  MAIN_ny82bdpadgnqmy8l.seg	MAIN_WRITELOCK


## TF-IDF

In [20]:
ind = index.open_dir("indCranfieldPSRemoveStop")
searcher = ind.searcher(weighting=scoring.TF_IDF())
parser = qparser.QueryParser("content", ind.schema, group=qparser.OrGroup)

results_dict = {}

start_time = time.time()

for query_name, query_text in queries.items():
    temp = preprocessQueryStemmingAnalyzer(query_text)
    query = parser.parse(temp)
    num_values = len(trueResult)
    results = searcher.search(query, limit=num_values)
    docid_list = [int(result['docid']) for result in results]
    results_dict[query_name] = docid_list

total_timeStemRemoveStop_TFIDF = time.time() - start_time
print("Total execution time:", total_timeStemRemoveStop_TFIDF)

Total execution time: 3.4003987312316895


In [21]:
len(results_dict)

225

In [22]:
listAP_PSRemoveStop_TFIDF, listOfAP_PSRemoveStop_TFIDF_Inter, MAP_PSRemoveStop_TFIDF_Inter = MAP(trueResult, results_dict)
l_PSRemoveStop_TFIDF_p5, l_PSRemoveStop_TFIDF_r5, p5_PSRemoveStop_TFIDF, r5_PSRemoveStop_TFIDF = count_P_R(trueResult, results_dict)
listOfAP_PSRemoveStop_TFIDF_NonInter, MAP_PSRemoveStop_TFIDF_NonInter = MAP(trueResult, results_dict, False)

print(MAP_PSRemoveStop_TFIDF_Inter)
print(MAP_PSRemoveStop_TFIDF_NonInter)

0.04973493195243327
0.08383837176616088


In [23]:
print(p5_PSRemoveStop_TFIDF, r5_PSRemoveStop_TFIDF)

0.01660904418522613 0.3576911704597446


## BM25F

In [24]:
ind = index.open_dir("indCranfieldPSRemoveStop")
searcher = ind.searcher(weighting=scoring.BM25F())
parser = qparser.QueryParser("content", ind.schema, group=qparser.OrGroup)

results_dict = {}

start_time = time.time()

for query_name, query_text in queries.items():
    temp = preprocessQueryStemmingAnalyzer(query_text)
    query = parser.parse(temp)
    num_values = len(trueResult)
    results = searcher.search(query, limit=num_values)
    docid_list = [int(result['docid']) for result in results]
    results_dict[query_name] = docid_list

total_timeStemRemoveStop_BM25F = time.time() - start_time
print("Total execution time:", total_timeStemRemoveStop_BM25F)

Total execution time: 2.8890955448150635


In [25]:
len(results_dict)

225

In [26]:
listAP_PSRemoveStop_BM25F, listOfAP_PSRemoveStop_BM25F_Inter, MAP_PSRemoveStop_BM25F_Inter = MAP(trueResult, results_dict)
l_PSRemoveStop_BM25F_p5, l_PSRemoveStop_BM25F_r5, p5_PSRemoveStop_BM25F, r5_PSRemoveStop_BM25F = count_P_R(trueResult, results_dict)
listOfAP_PSRemoveStop_BM25F_NonInter, MAP_PSRemoveStop_BM25F_NonInter = MAP(trueResult, results_dict, False)

print(MAP_PSRemoveStop_BM25F_Inter)
print(MAP_PSRemoveStop_BM25F_NonInter)


0.06687369118633123
0.11503882576542694


In [27]:
print(p5_PSRemoveStop_BM25F, r5_PSRemoveStop_BM25F)

0.017162130604979214 0.37191311034209856


In [28]:
trec_11(listAP_PSRemoveStop_BM25F)

{0.0: 0.2133326408611154,
 0.1: 0.18653018335682392,
 0.2: 0.13340963206724515,
 0.3: 0.07526073775821855,
 0.4: 0.04720186126781434,
 0.5: 0.03528470981415779,
 0.6: 0.023953991771071197,
 0.7: 0.013212521691255918,
 0.8: 0.005127052876743746,
 0.9: 0.0011486357925986406,
 1.0: 0.0011486357925986406}

## Frequency

In [29]:
ind = index.open_dir("indCranfieldPSRemoveStop")
searcher = ind.searcher(weighting=scoring.Frequency())
parser = qparser.QueryParser("content", ind.schema, group=qparser.OrGroup)

results_dict = {}
start_time = time.time()
for query_name, query_text in queries.items():
    temp = preprocessQueryStemmingAnalyzer(query_text)
    query = parser.parse(temp)
    num_values = len(trueResult)
    results = searcher.search(query, limit=num_values)
    docid_list = [int(result['docid']) for result in results]
    results_dict[query_name] = docid_list

total_timeStemRemoveStop_Frequency = time.time() - start_time
print("Total execution time:", total_timeStemRemoveStop_Frequency)

Total execution time: 1.7468805313110352


In [30]:
len(results_dict)

225

In [31]:
listAP_PSRemoveStop_Frequency, listOfAP_PSRemoveStop_Frequency_Inter, MAP_PSRemoveStop_Frequency_Inter = MAP(trueResult, results_dict)
l_PSRemoveStop_Frequency_p5, l_PSRemoveStop_Frequency_r5, p5_PSRemoveStop_Frequency, r5_PSRemoveStop_Frequency = count_P_R(trueResult, results_dict)
listOfAP_PSRemoveStop_Frequency_NonInter, MAP_PSRemoveStop_Frequency_NonInter = MAP(trueResult, results_dict, False)

print(MAP_PSRemoveStop_Frequency_Inter)
print(MAP_PSRemoveStop_Frequency_NonInter)

0.041335875017462546
0.072123932755755


In [32]:
print(p5_PSRemoveStop_Frequency, r5_PSRemoveStop_Frequency)

0.01583867381485576 0.3334584308310513


# Stem Only

In [33]:
!mkdir /content/indCranfieldPS

In [34]:
# shutil.rmtree('/content/indCranfieldPS')

In [35]:
start_time = time.time()
schema = Schema(docid=STORED(), content=TEXT(stored=True, analyzer=StandardAnalyzer(stoplist=None)))
ix = create_in("indCranfieldPS", schema)
writer = ix.writer()

for i in range(1, len(sorted_data_dict)):
    writer.add_document(docid="{}".format(i + 1), content=sorted_data_dict[i])

writer.commit()
total_timeStem = time.time() - start_time
print("Total execution time:", total_timeStem)

Total execution time: 1.6804547309875488


In [36]:
!ls /content/indCranfieldPS

_MAIN_1.toc  MAIN_uxhyc4b625uhwt96.seg	MAIN_WRITELOCK


## TF-IDF

In [37]:
ind = index.open_dir("indCranfieldPS")
searcher = ind.searcher(weighting=scoring.TF_IDF())
parser = qparser.QueryParser("content", ind.schema, group=qparser.OrGroup)

results_dict = {}
start_time = time.time()

for query_name, query_text in queries.items():
    temp = preprocessQueryWithStopwords(query_text)
    query = parser.parse(temp)
    num_values = len(trueResult)
    results = searcher.search(query, limit=num_values)
    docid_list = [int(result['docid']) for result in results]
    results_dict[query_name] = docid_list
total_timeStem_TFIDF = time.time() - start_time
print("Total execution time:", total_timeStem_TFIDF)

Total execution time: 7.067417621612549


In [38]:
len(results_dict)

225

In [39]:
listAP_PS_TFIDF, listOfAP_PS_Inter, MAP_PS_Inter = MAP(trueResult, results_dict)
l_PS_p5, l_PS_r5, p5_PS, r5_PS = count_P_R(trueResult, results_dict)
listOfAP_PS_NonInter, MAP_PS_NonInter = MAP(trueResult, results_dict, False)

print(MAP_PS_Inter)
print(MAP_PS_NonInter)

0.019161125479212678
0.041793993074232795


In [40]:
print(p5_PS, r5_PS)

0.00867160493827161 0.22400859358690756


## BM25F

In [41]:
ind = index.open_dir("indCranfieldPS")
searcher = ind.searcher(weighting=scoring.BM25F())
parser = qparser.QueryParser("content", ind.schema, group=qparser.OrGroup)

results_dict = {}
start_time = time.time()

for query_name, query_text in queries.items():
    temp = preprocessQueryWithStopwords(query_text)
    query = parser.parse(temp)
    num_values = len(trueResult)
    results = searcher.search(query, limit=num_values)
    docid_list = [int(result['docid']) for result in results]
    results_dict[query_name] = docid_list
total_timeStem_BM25F = time.time() - start_time
print("Total execution time:", total_timeStem_BM25F)

Total execution time: 10.125821113586426


In [42]:
len(results_dict)

225

In [43]:
listAP_PS_BM25F, listOfAP_PS_BM25F_Inter, MAP_PS_BM25F_Inter = MAP(trueResult, results_dict)
l_PS_BM25F_p5, l_PS_BM25F_r5, p5_PS_BM25F, r5_PS_BM25F = count_P_R(trueResult, results_dict)
listOfAP_PS_BM25F_NonInter, MAP_PS_BM25F_NonInter = MAP(trueResult, results_dict, False)

print(MAP_PS_BM25F_Inter)
print(MAP_PS_BM25F_NonInter)

0.0602909723305534
0.10306447820029954


In [44]:
print(p5_PS_BM25F, r5_PS_BM25F)

0.013096296296296288 0.3519107906345158


## Frequency

In [45]:
ind = index.open_dir("indCranfieldPS")
searcher = ind.searcher(weighting=scoring.Frequency())
parser = qparser.QueryParser("content", ind.schema, group=qparser.OrGroup)

results_dict = {}
start_time = time.time()

for query_name, query_text in queries.items():
    temp = preprocessQueryWithStopwords(query_text)
    query = parser.parse(temp)

    num_values = len(trueResult)
    results = searcher.search(query, limit=num_values)

    docid_list = [int(result['docid']) for result in results]
    results_dict[query_name] = docid_list

total_timeStem_Frequency = time.time() - start_time
print("Total execution time:", total_timeStem_Frequency)

Total execution time: 4.553758382797241


In [46]:
len(results_dict)

225

In [47]:
listAP_PS_Frequency, listOfAP_PS_Frequency_Inter, MAP_PS_Frequency_Inter = MAP(trueResult, results_dict)
l_PS_Frequency_p5, l_PS_Frequency_r5, p5_PS_Frequency, r5_PS_Frequency = count_P_R(trueResult, results_dict)
listOfAP_PS_Frequency_NonInter, MAP_PS_Frequency_NonInter = MAP(trueResult, results_dict, False)

print(MAP_PS_Frequency_Inter)
print(MAP_PS_Frequency_NonInter)

0.008971689080643153
0.023151229818954976


In [48]:
print(p5_PS_Frequency, r5_PS_Frequency)

0.006755555555555557 0.1729824122756687


# Thống kê

In [49]:
import pandas as pd

# Define your data
q1 = ["Stem + Stop + TFIDF", total_timeStemRemoveStop, total_timeStemRemoveStop_TFIDF, MAP_PSRemoveStop_TFIDF_Inter, MAP_PSRemoveStop_TFIDF_NonInter, p5_PSRemoveStop_TFIDF, r5_PSRemoveStop_TFIDF]
q2 = ["Stem + Stop + BM25F", total_timeStemRemoveStop, total_timeStemRemoveStop_BM25F, MAP_PSRemoveStop_BM25F_Inter, MAP_PSRemoveStop_BM25F_NonInter, p5_PSRemoveStop_BM25F, r5_PSRemoveStop_BM25F]
q3 = ["Stem + Stop + Frequency", total_timeStemRemoveStop, total_timeStemRemoveStop_Frequency, MAP_PSRemoveStop_Frequency_Inter, MAP_PSRemoveStop_Frequency_NonInter, p5_PSRemoveStop_Frequency, r5_PSRemoveStop_Frequency]
q4 = ["Stem + TFIDF", total_timeStem, total_timeStem_TFIDF, MAP_PS_Inter, MAP_PS_NonInter, p5_PS, r5_PS]
q5 = ["Stem + BM25F", total_timeStem, total_timeStem_BM25F, MAP_PS_BM25F_Inter, MAP_PS_BM25F_NonInter, p5_PS_BM25F, r5_PS_BM25F]
q6 = ["Stem + Frequency", total_timeStem, total_timeStem_Frequency, MAP_PS_Frequency_Inter, MAP_PS_Frequency_NonInter, p5_PS_Frequency, r5_PS_Frequency]

col = ["Type", "Index Time", "Query Time", "Inter", "NonInter", "P", "R"]

# Create the DataFrame
df = pd.DataFrame([q1, q2, q3, q4, q5, q6], columns=col)

# Round the numeric columns to 2 decimal places
numeric_cols = ["Index Time", "Query Time", "Inter", "NonInter", "P", "R"]
df[numeric_cols] = df[numeric_cols].round(4)

# Print the rounded DataFrame
df


,Type,Index Time,Query Time,Inter,NonInter,P,R
0,Stem + Stop + TFIDF,3.7630,3.4004,0.0497,0.0838,0.0166,0.3577
1,Stem + Stop + BM25F,3.7630,2.8891,0.0669,0.1150,0.0172,0.3719
2,Stem + Stop + Frequency,3.7630,1.7469,0.0413,0.0721,0.0158,0.3335
3,Stem + TFIDF,1.6805,7.0674,0.0192,0.0418,0.0087,0.2240
4,Stem + BM25F,1.6805,10.1258,0.0603,0.1031,0.0131,0.3519
5,Stem + Frequency,1.6805,4.5538,0.0090,0.0232,0.0068,0.1730
